# Feature Engineering

## Cargue de librerias y parametros

In [26]:
%load_ext autoreload
%autoreload 2

import sys
import os

# Agrega la ruta del directorio 'src' al path
sys.path.append(os.path.abspath('../src'))

# Ahora puedes importar tus módulos
from procesamiento_datos import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
repo_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
datos_raw = "..//data/raw/"
datos_processed = "..//data/processed/"

## Cargue de los dataframes

In [3]:

# Cargar los datos
df_facturas = cargar_datos(datos_raw + "Historico_Facturas.csv")
df_cierres = cargar_datos(datos_raw + "Historico_Cierres.csv")
df_negocios = cargar_datos(datos_raw + "Tipos_Negocio.csv")
df_ipc = cargar_datos(datos_raw + "Historico_IPC.csv", sep=';')
df_tasas = cargar_datos(datos_raw + "Historico_Tasas_Credito.csv", sep=';')
df_pib = cargar_datos(datos_raw + "Historico_PIB_Corriente_Desc.csv")
df_sml = cargar_datos(datos_raw + "Historico_Salario_Minimo.csv")
df_clientes = cargar_datos(datos_raw + "Clientes_EEFF.csv")

# Pruebas

In [ ]:
convertir_fecha(df_facturas,['Fecha_expedicion','Fecha_pago','Fecha_vencimiento'])
df_facturas

In [ ]:
fecha_referencia = pd.to_datetime('2025-06-30')

def calcular_dias_demora(row):
    if row ['Estado'] == 'PAGADA' and row ['Fecha_pago'] != 'NaT':
        return ( row['Fecha_pago'] - row['Fecha_vencimiento']).days
    else:
        return ( fecha_referencia - row['Fecha_vencimiento']).days

df_facturas ['dias_demora'] = df_facturas.apply(calcular_dias_demora, axis=1)

In [ ]:
#si los días de mora son negativos o iguales a 30 el habito de pago es bueno = 1 , si son mayores a 30 el habito de pago es malo = 0 
df_facturas['habito_pago'] = df_facturas['dias_demora'].apply(lambda x: 1 if x <= 30 else 0)
df_facturas['ratio_pago'] =df_facturas['Valor_pagado']/df_facturas['Total factura']

In [ ]:
df_facturas_agg = df_facturas.copy()
df_facturas_agg = df_facturas_agg.groupby(['NIT']).agg(
    Total_facturado=('Total factura','sum'),
    Total_por_pagar = ('Saldo','sum'),
    Total_pagado = ('Valor_pagado','sum'),
    Cantidad_facturas= ('id_factura','count'),
    Dias_mora_promedio=('dias_demora','mean'),
    porcentaje_pagos_tiempo = ('habito_pago','mean'),
    promedio_ratio_pago = ('ratio_pago','mean'),
    desv_dias_demora = ('dias_demora','std'),
    maximo_dias_demora = ('dias_demora','max'),
    facturas_vencidas = ('Estado',lambda x: (x !="PAGADA").sum())
).reset_index()
df_facturas_agg.info()

In [ ]:
df_facturas_agg['Factura_promedio'] = round(df_facturas_agg['Total_facturado']/df_facturas_agg['Cantidad_facturas'])
df_facturas_agg.head()

## Creación de features

In [ ]:
# Crear sistema de scoring
def calcular_score_credito(row):
    score = 0
    
    # 1. Porcentaje de pagos a tiempo (max 30 puntos)
    score += min(row['porcentaje_pagos_tiempo'] * 0.3, 30)
    
    # 2. Días de mora promedio (max 25 puntos)
    if row['promedio_dias_mora'] <= 0:
        score += 25
    elif row['promedio_dias_mora'] <= 30:
        score += 15
    elif row['promedio_dias_mora'] <= 60:
        score += 5
    
    # 3. Ratio de pago promedio (max 20 puntos)
    score += min(row['ratio_pago_promedio'] * 20, 20)
    
    # 4. Facturas vencidas actuales (max 15 puntos)
    if row['facturas_vencidas_actuales'] == 0:
        score += 15
    elif row['facturas_vencidas_actuales'] <= 2:
        score += 7
    
    # 5. Volatilidad en pagos (max 10 puntos)
    if pd.isna(row['volatilidad_dias_mora']) or row['volatilidad_dias_mora'] <= 15:
        score += 10
    elif row['volatilidad_dias_mora'] <= 30:
        score += 5
    
    return score

# Calcular el score para cada cliente
indicadores_cliente['score_credito'] = indicadores_cliente.apply(calcular_score_credito, axis=1)

# Establecer categorías de riesgo basadas en el score
def categorizar_riesgo(score):
    if score >= 85:
        return 'A - Riesgo Bajo'
    elif score >= 70:
        return 'B - Riesgo Moderado Bajo'
    elif score >= 50:
        return 'C - Riesgo Moderado'
    elif score >= 30:
        return 'D - Riesgo Moderado Alto'
    else:
        return 'E - Riesgo Alto'

indicadores_cliente['categoria_riesgo'] = indicadores_cliente['score_credito'].apply(categorizar_riesgo)

# Mostrar distribución de scores y categorías
print("\nDistribución de Scores de Crédito:")
print(indicadores_cliente['score_credito'].describe())

print("\nDistribución de Categorías de Riesgo:")
print(indicadores_cliente['categoria_riesgo'].value_counts())

## Incorporación de variables macroeconómicas
Agregaremos las variables macroeconómicas (IPC, PIB, tasas de crédito y salario mínimo) al DataFrame final. Estas variables son importantes para capturar el contexto económico en el que se desarrollan los comportamientos de pago.

In [29]:
df_sml = cargar_datos(datos_raw + "Historico_Salario_Minimo.csv")
df_sml.columns

Index(['Serie', 'Auxilio de transporte mensual', 'Salario mínimo mensual',
       'Salario mínimo mensual, sector rural',
       'Salario mínimo mensual, sector urbano alto'],
      dtype='object')

In [30]:
df_sml = df_sml.loc[:, ['Serie', 'Salario\xa0mínimo\xa0mensual']]
df_sml = df_sml.rename(columns={'Serie': 'fecha_cierre', 'Salario\xa0mínimo\xa0mensual': 'smmlv'})
convertir_fecha(df_sml, ['fecha_cierre'])
convertir_numerico(df_sml, ['smmlv'])
df_sml

,fecha_cierre,smmlv
0,1950-01-31,NaN
1,1950-02-28,NaN
2,1950-03-31,NaN
3,1950-04-30,NaN
4,1950-05-31,NaN
...,...,...
907,2025-08-31,1423500.0
908,2025-09-30,1423500.0
909,2025-10-31,1423500.0
910,2025-11-30,1423500.0


In [31]:
df_ipc = cargar_datos(datos_raw + "Historico_IPC.csv", sep=';')
df_ipc.columns

Index(['Fecha', 'Inflación núcleo 15 anual',
       'Inflación sin alimentos ni regulados, anual',
       'Inflación sin alimentos, anual'],
      dtype='object')

In [32]:
df_ipc = df_ipc.loc[:, ['Fecha', 'Inflación\xa0núcleo\xa015 anual']]
df_ipc = df_ipc.rename(columns={'Fecha':'fecha_cierre', 'Inflación\xa0núcleo\xa015 anual':'ipc'})
convertir_fecha(df_ipc, ['fecha_cierre'])
convertir_numerico(df_ipc, ['ipc'])

,fecha_cierre,ipc
0,1999-01-31,16.47
1,1999-02-28,15.35
2,1999-03-31,13.92
3,1999-04-30,12.98
4,1999-05-31,12.12
...,...,...
313,2025-02-28,5.42
314,2025-03-31,5.30
315,2025-04-30,5.34
316,2025-05-31,5.25


In [37]:
df_pib = cargar_datos(datos_raw + "Historico_PIB_Corriente_Desc.csv")
df_pib.columns

Index(['Fecha', '1. PIB reportado', '1.01. Valor agregado bruto',
       '1.01.01. Agricultura, ganadería, caza, silvicultura y pesca',
       '1.01.02. Explotación de minas y canteras',
       '1.01.03. Industrias manufactureras',
       '1.01.04. Suministro de electricidad, gas, vapor y\r\naire acondicionado (*)',
       '1.01.05. Construcción',
       '1.01.06. Comercio al por mayor y al por menor\r\n(*)',
       '1.01.07. Información y comunicaciones',
       '1.01.08. Actividades financieras y de seguros',
       '1.01.09. Actividades inmobiliarias',
       '1.01.10. Actividades profesionales, científicas y\r\ntécnicas (*)',
       '1.01.11. Administración pública, defensa,\r\neducación y salud (*)',
       '1.01.12. Actividades artísticas, de\r\nentretenimiento y recreación y otras actividades\r\nde servicios (*)',
       '1.02. Impuestos menos subvenciones sobre los\r\nproductos',
       ' \r\nLos valores ausentes se indican con un punto (.)\r\nDescargado del sistema del Banco d

In [38]:
df_pib = df_pib = df_pib.loc[:, ['Fecha', '1. PIB reportado']]
df_pib = df_pib.rename(columns={'Fecha':'fecha_cierre','1. PIB reportado': 'pib'})
convertir_fecha(df_pib, ['fecha_cierre'])
convertir_numerico(df_pib, ['pib'])

,fecha_cierre,pib
0,2005-01-31,81365.11
1,2005-02-28,81365.11
2,2005-03-31,81365.11
3,2005-04-30,84193.31
4,2005-05-31,84193.31
...,...,...
238,2024-11-30,445607.92
239,2024-12-31,445607.92
240,2025-01-31,452132.02
241,2025-02-28,452132.02


In [42]:
df_tasas = cargar_datos(datos_raw + "Historico_Tasas_Credito.csv",sep=';')
df_tasas.columns

Index(['Fecha', 'Créditos de consumo', 'Créditos de tesorería',
       'Créditos ordinarios', 'Créditos preferenciales',
       'Tasas de Colocación Banco de la República',
       'Tasas de Colocación sin tesoreria', 'Tasas de Colocación total'],
      dtype='object')

In [44]:
df_tasas = df_tasas.loc[:,['Fecha','Créditos\xa0de\xa0consumo']]
df_tasas = df_tasas.rename(columns={'Fecha':'fecha_cierre','Créditos\xa0de\xa0consumo':'tasa'})
convertir_fecha(df_tasas, ['fecha_cierre'])
convertir_numerico(df_tasas, ['tasa'])

,fecha_cierre,tasa
0,1998-03-31,42.24
1,1998-04-30,44.08
2,1998-05-31,45.32
3,1998-06-30,48.41
4,1998-07-31,51.57
...,...,...
323,2025-02-28,19.40
324,2025-03-31,19.35
325,2025-04-30,19.51
326,2025-05-31,19.29


In [45]:
df_variables_macro = df_sml.merge(df_ipc, on='fecha_cierre', how= 'left')
df_variables_macro = df_variables_macro.merge(df_pib, on='fecha_cierre',how='left')
df_variables_macro = df_variables_macro.merge(df_tasas[['fecha_cierre', 'tasa']], on='fecha_cierre', how='left')
df_variables_macro.head()

,fecha_cierre,smmlv,ipc,pib,tasa
0,1950-01-31,NaN,NaN,NaN,NaN
1,1950-02-28,NaN,NaN,NaN,NaN
2,1950-03-31,NaN,NaN,NaN,NaN
3,1950-04-30,NaN,NaN,NaN,NaN
4,1950-05-31,NaN,NaN,NaN,NaN


In [48]:
def variaciones_porcentuales(df,columnas:list, periodos: int=1):
    for columna in columnas:
        nueva_columna = f'var_pct_{columna}_{periodos}m'
        df[nueva_columna] = df[columna].pct_change(periods=periodos) * 100 

variaciones_porcentuales(df_variables_macro, ['ipc','pib','tasa','smmlv'], periodos=1)


In [49]:
df_variables_macro.columns

Index(['fecha_cierre', 'smmlv', 'ipc', 'pib', 'tasa', 'var_pct_ipc_1m',
       'var_pct_pib_1m', 'var_pct_tasa_1m', 'var_pct_smmlv_1m'],
      dtype='object')

In [50]:
columnas = ['var_pct_ipc_1m','var_pct_pib_1m','var_pct_tasa_1m','var_pct_smmlv_1m']

for col in columnas:
    df_variables_macro[col] = df_variables_macro[col].fillna(0).replace(0, method='ffill')

df_variables_macro

,fecha_cierre,smmlv,ipc,pib,tasa,var_pct_ipc_1m,var_pct_pib_1m,var_pct_tasa_1m,var_pct_smmlv_1m
0,1950-01-31,NaN,NaN,NaN,NaN,0.00000,0.00000,0.000000,0.0
1,1950-02-28,NaN,NaN,NaN,NaN,0.00000,0.00000,0.000000,0.0
2,1950-03-31,NaN,NaN,NaN,NaN,0.00000,0.00000,0.000000,0.0
3,1950-04-30,NaN,NaN,NaN,NaN,0.00000,0.00000,0.000000,0.0
4,1950-05-31,NaN,NaN,NaN,NaN,0.00000,0.00000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...
907,2025-08-31,1423500.0,NaN,NaN,NaN,-2.47619,1.46409,-1.192328,9.5
908,2025-09-30,1423500.0,NaN,NaN,NaN,-2.47619,1.46409,-1.192328,9.5
909,2025-10-31,1423500.0,NaN,NaN,NaN,-2.47619,1.46409,-1.192328,9.5
910,2025-11-30,1423500.0,NaN,NaN,NaN,-2.47619,1.46409,-1.192328,9.5


In [51]:
exportar_csv(df_variables_macro, datos_raw + "Variables_Macro.csv")

Datos exportados exitosamente a ..//data/raw/Variables_Macro.csv


## Definición de la variable objetivo (Default)
Crearemos la variable objetivo basada en dos criterios:
1. Días de mora > 90 días
2. Calificación crediticia (D o E)

In [ ]:
# Definir default basado en días de mora y calificación
df_final['default'] = ((df_final['dias_mora'] > 90) | 
                      (df_final['calificacion'].isin(['D', 'E']))).astype(int)

# Calcular la tasa de default
tasa_default = df_final['default'].mean() * 100

print(f"Tasa de default en la cartera: {tasa_default:.2f}%")

# Mostrar distribución de defaults por categoría de riesgo
print("\nDistribución de defaults por categoría de riesgo:")
print(pd.crosstab(df_final['categoria_riesgo'], df_final['default'], 
                  normalize='index') * 100)

# Guardar el DataFrame final procesado
df_final.to_csv('../data/processed/datos_modelado.csv', index=False)
print("\nDataset guardado en '../data/processed/datos_modelado.csv'")

In [ ]:
df_final